**BNN**

In [5]:
from utils import *
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler
import h5py
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset, Subset
quant = 'BNN'
class Q_CAD(nn.Module):
    def __init__(self):
        super(Q_CAD, self).__init__()
        self.conv1 = QConv2d(in_channels=1, out_channels=64, kernel_size=(3, 3), padding=1, quant=quant)
        self.conv2 = QConv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), padding=1, quant=quant)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = QLinear(in_features=64 * int(32/4) * 2, out_features=128, quant=quant)
        self.fc2 = QLinear(in_features=128, out_features=2, quant=quant)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x


model = Q_CAD()
model.eval()
model.load_state_dict(torch.load('/home/usami/TBN64/train/model_save/BNN3032.pth'))

<All keys matched successfully>

In [8]:
num_testcase = 1000
with open('../testcase_bnn.bin', 'wb') as f:
    np.array([num_testcase], dtype=np.int32).tofile(f)
    for i in range(num_testcase):
        input = torch.rand(1,1,32,9)
        i = input
        predict = model(input).data
        input = input.numpy()
        predict = predict.numpy()
        input.tofile(f)
        predict.tofile(f)

In [3]:
def _binary(x: torch.Tensor, delta: float):
    return (x >= delta).float() - (x < delta).float()
with open('../model_bnn.bin', 'wb') as f:
    for name, module in model.named_modules():
        # print(module)
        if isinstance(module, QConv2d):
            weight_shape = module.weight.data.shape
            input_channel = weight_shape[1]
            output_channel = weight_shape[0]
            weight = module.weight.data
            input_quant = module.input_quantizer.state_dict()['running_delta']
            weight_quant = module.weight_quantizer.state_dict()['running_delta']
        
            weight_q = np.array(_binary(weight, weight_quant))
            weight_q[weight_q == 1] = 0
            weight_q[weight_q == -1] = 1
            weight_qc = np.zeros((weight_shape[0], int(np.ceil(weight_shape[1]/64)), weight_shape[2] , weight_shape[3]), dtype=np.int64)


            for oc in range(output_channel):
                for ic in range(input_channel):
                    for kx in range(weight_shape[2]):
                        for ky in range(weight_shape[3]):
                            weight_qc[oc][int(np.floor(ic/64))][kx][ky] |= np.int64(weight_q[oc][ic][kx][ky]) << (ic%64)
            # weight_qc = weight_qc.view(-1)
            np.array([1], dtype=np.int32).tofile(f) # save CONV
            np.array([weight_shape[0]], dtype=np.int32).tofile(f) # save output channels
            np.array([int(np.ceil(weight_shape[1]/64))], dtype=np.int32).tofile(f) # save input channels
            np.array([weight_shape[2]], dtype=np.int32).tofile(f) # save kernel size x
            np.array([weight_shape[3]], dtype=np.int32).tofile(f) # save kernel size y

            np.array([input_quant], dtype=np.float32).tofile(f)
            # np.array([weight_quant], dtype=np.float32).tofile(f)
            np.array(weight_qc, dtype=np.int64).tofile(f)
            # break
        elif isinstance(module, QLinear):
            # break
            weight_shape = module.weight.data.shape
            input_channel = weight_shape[1]
            output_channel = weight_shape[0]
            weight = module.weight.data
            input_quant = module.input_quantizer.state_dict()['running_delta']
            weight_quant = module.weight_quantizer.state_dict()['running_delta']
        
            weight_q = np.array(_binary(weight, weight_quant))
            weight_q[weight_q == 1] = 0
            weight_q[weight_q == -1] = 1
            

            weight_qc = np.zeros((weight_shape[0], int(np.ceil(weight_shape[1]/64))), dtype=np.int64)
            for oc in range(output_channel):
                for ic in range(input_channel):
                            weight_qc[oc][int(np.floor(ic/64))] |= np.int64(weight_q[oc][ic]) << (ic%64)
            # weight_qc = weight_qc.view(-1)
            np.array([0], dtype=np.int32).tofile(f) # save LINEAR
            np.array([weight_shape[0]], dtype=np.int32).tofile(f) # save output channels
            np.array([int(np.ceil(weight_shape[1]/64))], dtype=np.int32).tofile(f) # save input channels

            np.array([input_quant], dtype=np.float32).tofile(f)
            # np.array([weight_quant], dtype=np.float32).tofile(f)
            np.array(weight_qc, dtype=np.int64).tofile(f)
            # break


/tmp/ipykernel_3003/2666181971.py:32: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  np.array([input_quant], dtype=np.float32).tofile(f)
/tmp/ipykernel_3003/2666181971.py:59: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np

***TBN***

In [142]:
from utils import *
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler
import h5py
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset, Subset
def _ternary(x: torch.Tensor, delta: float):
    return (x >= delta).float() - (x <= -delta).float()
quant = 'TBN'
class Q_CAD(nn.Module):
    def __init__(self):
        super(Q_CAD, self).__init__()
        self.conv1 = QConv2d(in_channels=1, out_channels=64, kernel_size=(3, 3), padding=1, quant=quant)
        self.conv2 = QConv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), padding=1, quant=quant)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = QLinear(in_features=64 * int(32/4) * 2, out_features=128, quant=quant)
        self.fc2 = QLinear(in_features=128, out_features=2, quant=quant)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x


model = Q_CAD()
model.eval()
model.load_state_dict(torch.load('/home/usami/TBN64/train/model_save/TBN3032.pth'))

<All keys matched successfully>

In [133]:
def _binary(x: torch.Tensor, delta: float):
    return (x >= delta).float() - (x < delta).float()
with open('../model_tbn.bin', 'wb') as f:
    for name, module in model.named_modules():
        # print(module)
        if isinstance(module, QConv2d):
            weight_shape = module.weight.data.shape
            input_channel = weight_shape[1]
            output_channel = weight_shape[0]
            weight = module.weight.data
            input_quant = module.input_quantizer.state_dict()['running_delta']
            weight_quant = module.weight_quantizer.state_dict()['running_delta']
        
            weight_q = np.array(_binary(weight, weight_quant))
            weight_q[weight_q == 1] = 0
            weight_q[weight_q == -1] = 1
            weight_qc = np.zeros((weight_shape[0], int(np.ceil(weight_shape[1]/64)), weight_shape[2] , weight_shape[3]), dtype=np.int64)


            for oc in range(output_channel):
                for ic in range(input_channel):
                    for kx in range(weight_shape[2]):
                        for ky in range(weight_shape[3]):
                            weight_qc[oc][int(np.floor(ic/64))][kx][ky] |= np.int64(weight_q[oc][ic][kx][ky]) << (ic%64)
            # weight_qc = weight_qc.view(-1)
            np.array([1], dtype=np.int32).tofile(f) # save CONV
            np.array([weight_shape[0]], dtype=np.int32).tofile(f) # save output channels
            np.array([int(np.ceil(weight_shape[1]/64))], dtype=np.int32).tofile(f) # save input channels
            np.array([weight_shape[2]], dtype=np.int32).tofile(f) # save kernel size x
            np.array([weight_shape[3]], dtype=np.int32).tofile(f) # save kernel size y

            np.array([input_quant], dtype=np.float32).tofile(f)
            np.array(weight_qc, dtype=np.int64).tofile(f)
        elif isinstance(module, QLinear):
            weight_shape = module.weight.data.shape
            input_channel = weight_shape[1]
            output_channel = weight_shape[0]
            weight = module.weight.data
            input_quant = module.input_quantizer.state_dict()['running_delta']
            weight_quant = module.weight_quantizer.state_dict()['running_delta']
        
            weight_q = np.array(_binary(weight, weight_quant))
            weight_q[weight_q == 1] = 0
            weight_q[weight_q == -1] = 1
            

            weight_qc = np.zeros((weight_shape[0], int(np.ceil(weight_shape[1]/64))), dtype=np.int64)
            for oc in range(output_channel):
                for ic in range(input_channel):
                            weight_qc[oc][int(np.floor(ic/64))] |= np.int64(weight_q[oc][ic]) << (ic%64)
            # weight_qc = weight_qc.view(-1)
            np.array([0], dtype=np.int32).tofile(f) # save LINEAR
            np.array([weight_shape[0]], dtype=np.int32).tofile(f) # save output channels
            np.array([int(np.ceil(weight_shape[1]/64))], dtype=np.int32).tofile(f) # save input channels

            np.array([input_quant], dtype=np.float32).tofile(f)
            np.array(weight_qc, dtype=np.int64).tofile(f)


/tmp/ipykernel_3179/967514026.py:32: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  np.array([input_quant], dtype=np.float32).tofile(f)
/tmp/ipykernel_3179/967514026.py:59: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.e

In [148]:
num_testcase = 1000
with open('../testcase_tbn.bin', 'wb') as f:
    np.array([num_testcase], dtype=np.int32).tofile(f)
    for i in range(num_testcase):
        input = torch.rand(1,1,32,9)
        i = input
        predict = model(input).data
        input = input.numpy()
        predict = predict.numpy()
        input.tofile(f)
        predict.tofile(f)
# print("INPUT: ")
# print(input)
# print("OUTPUT: ")
# print(predict)

****TNN****

In [187]:
from utils import *
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler
import h5py
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset, Subset

quant = 'TNN'
class Q_CAD(nn.Module):
    def __init__(self):
        super(Q_CAD, self).__init__()
        self.conv1 = QConv2d(in_channels=1, out_channels=64, kernel_size=(3, 3), padding=1, quant=quant)
        self.conv2 = QConv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), padding=1, quant=quant)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = QLinear(in_features=64 * int(32/4) * 2, out_features=128, quant=quant)
        self.fc2 = QLinear(in_features=128, out_features=2, quant=quant)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.pool(x)
        x = self.conv2(x)
        print(x[0][0])
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x


model = Q_CAD()
model.eval()
model.load_state_dict(torch.load('/home/usami/TBN64/train/model_save/TNN3032.pth'))

<All keys matched successfully>

In [189]:
def _ternary(x: torch.Tensor, delta: float):
    return (x >= delta).float() - (x <= -delta).float()
with open('../model_tnn.bin', 'wb') as f:
    for name, module in model.named_modules():
        # print(module)
        if isinstance(module, QConv2d):
            weight_shape = module.weight.data.shape
            input_channel = weight_shape[1]
            output_channel = weight_shape[0]
            weight = module.weight.data
            input_quant = module.input_quantizer.state_dict()['running_delta']
            weight_quant = module.weight_quantizer.state_dict()['running_delta']
        
            weight_q = np.array(_ternary(weight, weight_quant))
            weight_qc_bit0 = np.zeros((weight_shape[0], int(np.ceil(weight_shape[1]/64)), weight_shape[2] , weight_shape[3]), dtype=np.int64)
            weight_qc_bit1 = np.zeros((weight_shape[0], int(np.ceil(weight_shape[1]/64)), weight_shape[2] , weight_shape[3]), dtype=np.int64)

            for oc in range(output_channel):
                for ic in range(input_channel):
                    for kx in range(weight_shape[2]):
                        for ky in range(weight_shape[3]):
                            tmp_weight_q = weight_q[oc][ic][kx][ky]
                            if(tmp_weight_q == -1):
                                weight_qc_bit0[oc][int(np.floor(ic/64))][kx][ky] |= np.int64(1) << (ic%64)
                            elif(tmp_weight_q == 1):
                                weight_qc_bit1[oc][int(np.floor(ic/64))][kx][ky] |= np.int64(1) << (ic%64)
                            else:  
                                pass
            # weight_qc = weight_qc.view(-1)
            np.array([1], dtype=np.int32).tofile(f) # save CONV
            np.array([weight_shape[0]], dtype=np.int32).tofile(f) # save output channels
            np.array([int(np.ceil(weight_shape[1]/64))], dtype=np.int32).tofile(f) # save input channels
            np.array([weight_shape[2]], dtype=np.int32).tofile(f) # save kernel size x
            np.array([weight_shape[3]], dtype=np.int32).tofile(f) # save kernel size y

            np.array([input_quant], dtype=np.float32).tofile(f)
            # np.array([weight_quant], dtype=np.float32).tofile(f)
            np.array(weight_qc_bit0, dtype=np.int64).tofile(f)
            np.array(weight_qc_bit1, dtype=np.int64).tofile(f)
            # break
        elif isinstance(module, QLinear):
            weight_shape = module.weight.data.shape
            input_channel = weight_shape[1]
            output_channel = weight_shape[0]
            weight = module.weight.data
            input_quant = module.input_quantizer.state_dict()['running_delta']
            weight_quant = module.weight_quantizer.state_dict()['running_delta']
        
            weight_q = np.array(_ternary(weight, weight_quant))
        
            weight_qc_bit0 = np.zeros((weight_shape[0], int(np.ceil(weight_shape[1]/64))), dtype=np.int64)
            weight_qc_bit1 = np.zeros((weight_shape[0], int(np.ceil(weight_shape[1]/64))), dtype=np.int64)

            for oc in range(output_channel):
                for ic in range(input_channel):
                    tmp_weight_q = weight_q[oc][ic]
                    if(tmp_weight_q == -1):
                        weight_qc_bit0[oc][int(np.floor(ic/64))] |= np.int64(1) << (ic%64)
                    if(tmp_weight_q == 1):
                        weight_qc_bit1[oc][int(np.floor(ic/64))] |= np.int64(1) << (ic%64)
                    else:
                        pass
            # weight_qc = weight_qc.view(-1)
            np.array([0], dtype=np.int32).tofile(f) # save LINEAR
            np.array([weight_shape[0]], dtype=np.int32).tofile(f) # save output channels
            np.array([int(np.ceil(weight_shape[1]/64))], dtype=np.int32).tofile(f) # save input channels

            np.array([input_quant], dtype=np.float32).tofile(f)
            # np.array([weight_quant], dtype=np.float32).tofile(f)
            np.array(weight_qc_bit0, dtype=np.int64).tofile(f)
            np.array(weight_qc_bit1, dtype=np.int64).tofile(f)
            # break


/tmp/ipykernel_3179/880937725.py:36: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  np.array([input_quant], dtype=np.float32).tofile(f)
/tmp/ipykernel_3179/880937725.py:68: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.e

In [190]:
num_testcase = 1000
with open('../testcase_tnn.bin', 'wb') as f:
    np.array([num_testcase], dtype=np.int32).tofile(f)
    for i in range(num_testcase):
        input = torch.rand(1,1,32,9)
        i = input
        predict = model(input).data
        input = input.numpy()
        predict = predict.numpy()
        input.tofile(f)
        predict.tofile(f)
# print("INPUT: ")
# print(input)
# print("OUTPUT: ")
# print(predict)

tensor([[ -8., -19., -12., -11.],
        [-13., -14.,  -5.,   7.],
        [-18., -27.,  -8.,   8.],
        [-16., -17.,  -3.,   8.],
        [-11., -10.,   1.,   8.],
        [ -9.,  -7.,   1.,   8.],
        [ -9.,  -7.,   1.,   8.],
        [ -9.,  -7.,   1.,   8.],
        [ -9.,  -7.,   1.,   8.],
        [ -9.,  -7.,   1.,   8.],
        [ -9.,  -7.,   1.,   8.],
        [ -9.,  -7.,   1.,   8.],
        [ -9.,  -7.,   1.,   8.],
        [ -9.,  -7.,   1.,   8.],
        [-11.,  -6.,   1.,   5.],
        [-20., -24., -13.,   0.]], grad_fn=<SelectBackward0>)
tensor([[-10., -21., -16., -11.],
        [-15., -14.,  -2.,   7.],
        [-11., -10.,   1.,   8.],
        [ -9.,  -7.,   1.,   8.],
        [-15., -10.,   1.,   8.],
        [-12., -11.,   1.,   8.],
        [-11.,  -9.,   0.,  10.],
        [ -9.,  -7.,  -3.,   0.],
        [ -9., -11.,  -9., -13.],
        [-15.,  -9.,  -2.,   3.],
        [-12., -11.,   1.,   8.],
        [-11., -10.,   1.,   8.],
        [ -9.,  -7.,